In [1]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

#'Alessi_36', 'Collinder_135', 'NGC_2451B', 'OC_0450' Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

A new version of galpy (1.10.1) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [2]:
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
#df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

df_hunt_good = df_hunt_full.loc[
    (df_hunt_full['n_rv_gaia_apogee_galah'] >= 5) & 
    (df_hunt_full['U_err'] < 5) & 
    (df_hunt_full['V_err'] < 5) & 
    (df_hunt_full['W_err'] < 5) &
    (df_hunt_full['x'].between(-1000, 1000)) &
    (df_hunt_full['y'].between(-1000, 1000)) &
    (df_hunt_full['z'].between(-300, 300)) &
    (df_hunt_full['class_50'] >0.5) &
    (df_hunt_full['cst'] > 5)
    #(df_hunt_full['n_stars'] > 100)
    ]
df_hunt_young = df_hunt_good.loc[df_hunt_good['age_myr'] < 15]
df_hunt_older = df_hunt_good.loc[df_hunt_good['age_myr'].between(70, 150)]
df_hunt_older_2 = df_hunt_good.loc[df_hunt_good['age_myr'].between(150, 1000)]
df_hunt_older.to_csv('/Users/cam/Downloads/hunt_catalog_0_70_myr.csv', index=False)
df_hunt_young.to_csv('/Users/cam/Downloads/hunt_catalog_0_15_myr.csv', index=False)

In [3]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
#df_hunt_gagne = df_hunt_gagne.loc[df_hunt_gagne['n_stars'] >= 100]

# # CHRONOS ages
# df_hunt_new_ages = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages_2.csv')
# df_hunt_gagne = df_hunt_gagne.merge(df_hunt_new_ages, on='name', how='left', suffixes=('', '_new_ages'))

# # use 'age_chronos_mode' for age if it exists, otherwise use 'age_myr'
# df_hunt_gagne['age'] = df_hunt_gagne['age_chronos_mode']
# df_hunt_gagne.loc[df_hunt_gagne['age'].isnull(), 'age'] = df_hunt_gagne['age_myr']
# df_hunt_gagne['age_myr'] = df_hunt_gagne['age']
# df_hunt_gagne.to_csv('/Users/cam/Downloads/cluster_sample_data_chornos_ages.csv', index=False)

df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [27.], 'n_stars' : [1]})


In [4]:
g1_names = pd.read_csv('/Users/cam/Downloads/g1.csv')
g2_names = pd.read_csv('/Users/cam/Downloads/g2.csv')
g3_names = pd.read_csv('/Users/cam/Downloads/g3.csv')
g1 = df_hunt_full.loc[df_hunt_full['name'].isin(g1_names['name'])]
g2 = df_hunt_full.loc[df_hunt_full['name'].isin(g2_names['name'])]
g3 = df_hunt_full.loc[df_hunt_full['name'].isin(g3_names['name'])]

g1_data = StarClusterData(g1, data_name = 'Sirius MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)
g2_data = StarClusterData(g2, data_name = 'Coma MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)
g3_data = StarClusterData(g3, data_name = 'Pleiades MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)


In [5]:
import random

#df_hdbscan = pd.read_csv('/Users/cam/Downloads/clusters_hdbscan_results.csv')
#df_hdbscan = pd.read_csv('/Users/cam/Downloads/clusters_hdbscanned_dec21.csv')
df_hdbscan = pd.read_csv('/Users/cam/Downloads/clusters_hdbscanned_jan6.csv')
# Group by 'final_label' and create a new StarClusterData object for each group
grouped = df_hdbscan.groupby('final_label')
cluster_data_objects = []

for label, group in grouped:
    if label == -1:
        continue
    color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    cluster_data = StarClusterData(group, data_name=f'Cluster {label}', min_size=0, max_size=9, color=color, opacity=1., marker_style='circle')
    cluster_data_objects.append(cluster_data)

# Dark plot

In [6]:
# families
sample_data = StarClusterData(df_hunt_gagne, data_name = 'Clean Clusters Sample', min_size = 0, max_size = 9, color = 'gray', opacity = 1., marker_style = 'circle')
ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 9, color = 'violet', opacity = 1, marker_style = 'circle',  show_tracks = False)
m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 9, color = 'cyan', opacity = 1, marker_style = 'circle',  show_tracks = False)
cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 9, color = 'orange', opacity = 1, marker_style = 'circle',  show_tracks = False)
gamma_vel_data = StarClusterData(gamma_vel, data_name = 'Gamma Vel', min_size = 0, max_size = 9, color = 'red', opacity = 1., marker_style = 'circle')

young_data = StarClusterData(df_hunt_young, data_name = 'Clusters (< 15 Myr)', min_size = 0., max_size = 9, color = 'white', opacity = 0.5, marker_style = 'circle')
older_data = StarClusterData(df_hunt_older, data_name = 'Clusters (Age = (0, 200 Myr))', min_size = 0., max_size = 5, color = 'gray', opacity = 1., marker_style = 'circle')
older_data_2 = StarClusterData(df_hunt_older_2, data_name = 'Clusters (Age = (150, 500 Myr))', min_size = 0., max_size = 3, color = 'gray', opacity = 1., marker_style = 'circle')
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 8, color = 'yellow', opacity = 1., marker_style = 'circle')


#df_pleiades = df_hunt_good.loc[df_hunt_good['name'] == 'Melotte_22']
df_interesting_clusters = df_hunt_full.loc[df_hunt_full['name'].isin(['Melotte_22', 'Melotte_111', 'IC_4725', 'Melotte_25', 'NGC_2632', 'NGC_2516', 'Collinder_394', 'NGC_2287'])]
interesting_clusters = StarClusterData(df_interesting_clusters, data_name = 'Interesting Clusters', min_size = 0, max_size = 15, color = '#FF7F50', opacity = 1., marker_style = 'circle')

sc = StarClusterCollection([older_data, interesting_clusters] + cluster_data_objects)
grouping_dict = None

In [7]:
#xyz_widths = (1250, 1250, 400)
xyz_widths = (3000, 3000, 400)


plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    xyz_widths = xyz_widths, 
    #xyz_ranges = ([-10000, 10000], [-10000, 10000], [-500, 500]),
    figure_theme = 'dark', 
    trace_grouping_dict=grouping_dict)

save_name = '/Users/cam/Downloads/clustering_solution_visualize.html'
#save_name = '/Users/cam/Downloads/gal_movie_fade_in.html'
#save_name = '/Users/cam/Downloads/edenhofer_dust_movie.html'
#save_name = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/interactive_figures/edenhoder_dust_movie.html'
time_int = np.round(np.arange(0., -101, -1), 1)

fig3d = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = None, 
    static_traces_times = None, 
    static_traces_legendonly=True,
    focus_group = None,
    fade_in_time = 10, # Myr,
    fade_in_and_out = False,
    show_gc_line = True
)